# Optimization problems in explicit form

SciPy's optimization routines treat the objective function as a black box, so previously, I was able to define a funny loss function that had lots of if/then logic in it depending on whether a student was correctly classified.

However, by adding decision variables that correspond to the absolute difference between each misclassified students' score and the cutoff, we can create an explicit optimization problem. Moreover, the problem becomes a linear program if we assume $\rho = 1$, which (according to the results) seems to be a reasonable assumption.

Here I provide these optimization problems and validate them by showing that the results agree with those found in the main notebook.

I use Julia's JuMP package, which provides a more legible syntax for working with optimization problems that have many decision variables.

In [1]:
using JuMP
using Ipopt
using DataFrames
using CSV
ENV["COLUMNS"] = 100;

## Binary admissions outcomes

For the binary admissions data, where $y_n \in \left\{0, 1\right\}$ represents whether a student was admitted, we used an $L_1$ penalty function that adds up the absolute difference $\xi$ between the score and cutoff for any student who is misclassified. Using standard optimization tricks, it's not difficult to show that the problem formulated previously is equivalent to the following:

$$\begin{align}
\underset{a, \rho, p, \xi}{\text{minimize}}\quad&  \sum_{n} \xi_n \\[.75em]
\text{subject to}\quad & \Bigl(\sum_{t} a_t X_{nt}^\rho\Bigr)^{1/\rho} \leq p + \xi_n, \quad\forall n:  y_n = 0 \\[.5em]
 & \Bigl(\sum_{t} a_t X_{nt}^\rho\Bigr)^{1/\rho}\geq p - \xi_n, \quad \forall n: y_n = 1 \\[.5em]
 & \sum_t a_t = 1, \quad a,\,p,\,\xi \geq 0, \quad \rho \leq 1
\end{align}$$

Let's verify this empirically.

Read in the data.

In [2]:
binaryinp = DataFrame(CSV.File("binaryinp.csv"))
first(binaryinp, 5)

,Column1,admit,gre,gpa,rank,feat1,feat2,x1,x2
,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64
1,0,0,380,3.61,3,-210.867,-0.242558,0.0533333,0.473333
2,1,1,660,3.67,3,69.1331,-0.539923,0.703333,0.276667
3,2,1,800,4.0,1,209.135,1.31509,1.0,0.876667
4,3,1,640,3.19,4,49.1315,-1.52472,0.626667,0.06
5,4,0,520,2.93,4,-70.8687,-1.40029,0.253333,0.12


Formulate the optimization problem.

In [3]:
function optimize()
    N = size(binaryinp)[1]
    T = 2
    X = Array(binaryinp[!, [:x1, :x2]])
    
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    
    @variable(model, 0 ≤ a[1:T], start = 0.5)
    @variable(model, ρ ≤ 1, start = 0.5)
    @variable(model, 0 ≤ p, start = 0.5)
    @variable(model, 0 ≤ ξ[1:N])
    
    @objective(model, Min, sum(ξ))

    for n in 1:N
        if binaryinp[n, :admit] < 0.5            # If rejected
            @NLconstraint(model, sum(a[t] * X[n, t]^ρ for t in 1:T) ≤ (p + ξ[n])^ρ)
        else                                     # If admitted
            @NLconstraint(model, sum(a[t] * X[n, t]^ρ for t in 1:T) ≥ (p - ξ[n])^ρ)
        end
    end
    
    @constraint(model, Regularization, sum(a) == 1)
    
    optimize!(model)
    
    return value.(a), value.(ρ), value.(p)
end  

optimize (generic function with 1 method)

Notice that the results agree with those we found before:

In [4]:
a, ρ, p = optimize()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



([0.5167223716264429, 0.4832776283735572], 1.0, 0.5942642068479204)

## Probabilistic admissions outcomes

For probabilistic admissions data, $y_n \in \left[0, 1\right]$ represents each student's admissions probability. We weighted the $L_1$ penalty by how far our admissions "guess" (zero or one) was from the true probability. Again, using standard optimization tricks, the following problem is equivalent:

$$
\begin{align}
\underset{a, \rho, p, \xi, d}{\text{minimize}}\quad&  \sum_{n} d_n \\[.75em]
\text{subject to}\quad & \Bigl(\sum_{t} a_t X_{nt}^\rho\Bigr)^{1/\rho} - p = \xi_n \\[.5em]
			   & d_n \geq \xi_n\left(1 - y_n\right), \quad d_n \geq -\xi_n y_n  \\[.5em]
			   & \sum_t a_t = 1, \quad a,\,p,\,d \geq 0, \quad \rho \leq 1
\end{align}$$
The new variable
$$\begin{align}
d_n &\equiv 
\begin{cases}
 \xi_n \left(1-y_n\right), &  ~~\text{ if } u\left(X_{n.}\right)- p = \xi_n \geq 0 \\
-\xi_n y_n, & ~~\text{ if } u\left(X_{n.} \right) - p = \xi_n < 0
 \end{cases} \\[.5em]
  &= \max \left\{ \xi_n (1 - y_n), -\xi_n y_n \right\}
\end{align}$$
 represents how "correct" the guess is; notice no sign constraint on $\xi$. Again assuming $\rho = 1$ yields an LP. 
 
Let's check this.

Read in the data.

In [16]:
probinp = DataFrame(CSV.File("probinp.csv"))
probinp[1:5, [:chance, :x1, :x2]]

,chance,x1,x2
,Float64,Float64,Float64
1,0.92,0.965,0.5975
2,0.76,0.6675,0.1325
3,0.72,0.455,0.1725
4,0.8,0.6425,0.47
5,0.65,0.3825,0.1675


Optimization problem.

In [14]:
function optimize()
    N = size(probinp)[1]
    T = 2
    X = Array(probinp[!, [:x1, :x2]])
    y = Array(probinp[!, :chance])
    
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    
    @variable(model, 0 ≤ a[1:T], start = 0.5)
    @variable(model, ρ ≤ 1, start = 0.5)
    @variable(model, 0 ≤ p, start = 0.5)
    @variable(model, ξ[1:N])
    @variable(model, 0 ≤ d[1:N])
    
    @objective(model, Min, sum(d))

    @NLconstraint(model, Error[n in 1:N], sum(a[t] * X[n, t]^ρ for t in 1:T)^(1/ρ) == p + ξ[n])
    
    # This rearrangement yields a different solution that appears to be marginally better. 
    # But I am suspicious of it because it yields ρ ≈ 0, which yields low error in this equation
    # regardless of p and ξ.

    # @NLconstraint(model, Error[n in 1:N], sum(a[t] * X[n, t]^ρ for t in 1:T) == (p + ξ[n])^ρ)    
    
    @constraint(model, ErrorAdmit[n in 1:N], d[n] ≥ ξ[n] * (1 - y[n]))
    @constraint(model, ErrorReject[n in 1:N], d[n] ≥ -ξ[n] * y[n])
    @constraint(model, Regularization, sum(a) == 1)
    
    optimize!(model)
    
    @show objective_value(model)
    
    return value.(a), value.(ρ), value.(p)
end  

optimize (generic function with 1 method)

In [15]:
a, ρ, p = optimize()

objective_value(model) = 18.95084762846951


([0.6213902658254695, 0.37860973417453053], 0.9771543582677937, 0.35469610680026437)

The results agree with SciPy.